# Day 1

In [2]:
def two_sum(target, nums):
    data = set([target - val for val in nums])
    for val in nums:
        if val in data:
            print(val, target - val)

In [5]:
with open("p1_input") as fd:
    nums = [int(line.strip()) for line in fd]
    
two_sum(2020, nums)

1504 516
516 1504


In [6]:
1504 * 516

776064

In [20]:
def two_sum_v2(target, val, begin, nums, res):
    target -= val
    first, second = begin, len(nums) - 1
    while first < second:
        if nums[first] + nums[second] == target:
            res.append([val, nums[first], nums[second]])
            first += 1
            second -= 1
        elif nums[first] + nums[second] < target:
            first += 1
        elif nums[first] + nums[second] > target:
            second -= 1

def three_sum(target, nums):
    nums.sort()
    res = []
    for i, first in enumerate(nums):
        two_sum_v2(target, first, i + 1, nums, res)
    return res
        
res = three_sum(2020, nums)
print(res)
from functools import reduce
print(sum(res[0]), reduce(lambda x, y: x*y, res[0]))

[[13, 317, 1690]]
2020 6964490


In [21]:
13 * 317 * 1690

6964490

# Day 2

In [4]:
def valid_password(rule, pwd):
    cnt, symb = rule.split()
    min_cnt, max_cnt = map(int, cnt.split("-"))
    stats = {}
    for c in pwd:
        stats[c] = stats.get(c, 0) + 1
    return min_cnt <= stats.get(symb, 0) <= max_cnt


def valid_password2(rule, pwd):
    cnt, symb = rule.split()
    first, second = map(int, cnt.split("-"))
    count = int(pwd[first - 1] == symb) + int(pwd[second - 1] == symb)
    return count == 1


filename = "day2_passwords.txt"
data = [line.strip() for line in open(filename)]
cnt, cnt2 = 0, 0
for rec in data:
    rule, pwd = rec.split(":")
    cnt += int(valid_password(rule, pwd.strip()))
    cnt2 += int(valid_password2(rule, pwd.strip()))

print(cnt, cnt2)

493 593


# Day 3

In [4]:
def count_trees(board, dx=3, dy=1, tree="#"):
    maxx = len(board[0])
    maxy = len(board)
    x, y = 0, 0
    cnt = 0
    while y + dy < maxy:
        x = (x + dx) % maxx
        y += dy
        cnt += int(board[y][x] == tree)
    return cnt

filename = "day3_trees.txt"
data = []
with open(filename) as fd:
    for line in fd:
        data.append(list(line.strip()))

res = count_trees(data)
print(res)

slopes = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)] 
res = [count_trees(data, dx, dy) for dx, dy in slopes]
print(res)
import math
print(math.prod(res))

214
[94, 214, 99, 91, 46]
8336352024
